## Demonstrating KDtrees 
(loaded from lib/KDTreeEncoding)

In [1]:
%pylab inline
from numpy import load
from lib.KDTreeEncoding import *
from glob import glob

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Store location of current home directory
current=%pwd
#current

In [3]:
## List image files
import yaml
Yaml=yaml.safe_load(open('../config.yaml','r'))
_dir=Yaml['data_path']
files=glob(f'{_dir}*.npz')
len(files), files[:3]

(19669,
 ['/Users/yoavfreund/datasets/poverty_v1.1/anon_images/image18941.npz',
  '/Users/yoavfreund/datasets/poverty_v1.1/anon_images/image19487.npz',
  '/Users/yoavfreund/datasets/poverty_v1.1/anon_images/image6978.npz'])

## Train the tree on the files in "files"

In [4]:
%%time
train_size,tree=train_encoder(files[:100],tree_depth=7)

used 100 images to train KDTree
KDTree training data shape= (5017600, 8)
CPU times: user 1.42 s, sys: 128 ms, total: 1.55 s
Wall time: 1.55 s


## Print out a part of the tree

In [5]:
tree.print(tree.root.below.below,level=4)

00: size=1253519 index=6, threshold=  -0.87
001: size=626950 index=0, threshold=  -0.45
000: size=626569 index=4, threshold=  -0.69
0011: size=313509 index=4, threshold=  -0.29
0010: size=313441 index=3, threshold=  -0.40
0001: size=313414 index=0, threshold=  -0.26
0000: size=313155 index=5, threshold=  -1.06
00111: size=156868 index=7, threshold=  -0.17
00110: size=156641 index=1, threshold=  -0.38
00101: size=156870 index=6, threshold=  -0.44
00100: size=156571 index=6, threshold=  -0.48
00011: size=157121 index=0, threshold=   0.15
00010: size=156293 index=2, threshold=  -0.48
00001: size=180596 index=1, threshold=  -0.55
00000: size=132559 index=2, threshold=  -1.10


In [6]:
table='../public_tables/train.csv'
!head $table

,filename,country,wealthpooled,urban,label,nl_mean
1,image7716.npz,21,-1.0900524854660034,False,0,-0.0976840257644653
4,image603.npz,21,1.4540644884109497,True,1,11.082343101501465
5,image1199.npz,21,1.70844566822052,True,1,12.646743774414062
6,image2826.npz,21,1.701109528541565,True,1,5.055367469787598
7,image6082.npz,21,1.5477131605148315,True,1,12.907397270202637
8,image13809.npz,21,1.5924626588821411,True,1,13.152287483215332
9,image9062.npz,21,1.7062891721725464,True,1,13.142542839050291
11,image5804.npz,21,0.9911735653877258,True,0,6.250482082366943
12,image15372.npz,21,1.166226148605347,True,0,6.056130409240723


In [7]:
import pandas as pd

T=pd.read_csv(f'{current}/{table}')
T

,Unnamed: 0,filename,country,wealthpooled,urban,label,nl_mean
0,1,image7716.npz,21,-1.090052,False,0,-0.097684
1,4,image603.npz,21,1.454064,True,1,11.082343
2,5,image1199.npz,21,1.708446,True,1,12.646744
3,6,image2826.npz,21,1.701110,True,1,5.055367
4,7,image6082.npz,21,1.547713,True,1,12.907397
...,...,...,...,...,...,...,...
10905,19664,image9030.npz,5,1.159348,True,0,1.005626
10906,19665,image11611.npz,5,1.268232,True,0,0.895127
10907,19666,image12646.npz,5,1.506693,True,1,1.183905
10908,19667,image17629.npz,5,1.882921,True,1,0.515834


In [8]:
rural_5=pd.DataFrame(T[(T['country']==5) & (T['urban']==False)])
rural_5.head(2)

,Unnamed: 0,filename,country,wealthpooled,urban,label,nl_mean
10327,18876,image1149.npz,5,-0.467258,False,1,-0.173862
10328,18877,image6388.npz,5,-0.714008,False,0,-0.173862


In [9]:
rural_5['rich']=rural_5['wealthpooled']>-0.5
rural_5.index=rural_5.filename
rural_5.head()

,Unnamed: 0,filename,country,wealthpooled,urban,label,nl_mean,rich
filename,,,,,,,,
image1149.npz,18876,image1149.npz,5,-0.467258,False,1,-0.173862,True
image6388.npz,18877,image6388.npz,5,-0.714008,False,0,-0.173862,False
image15364.npz,18889,image15364.npz,5,-0.628777,False,0,-0.173862,False
image10130.npz,18908,image10130.npz,5,0.937543,False,1,0.029008,True
image3478.npz,18911,image3478.npz,5,-0.117946,False,1,-0.172251,True


In [10]:
rural_5.shape,_dir

((342, 8), '/Users/yoavfreund/datasets/poverty_v1.1/anon_images/')

In [11]:
%%time
EC=encoded_dataset(_dir[:-1],rural_5,tree)

CPU times: user 3.82 s, sys: 841 ms, total: 4.66 s
Wall time: 4.65 s


In [12]:
EC.data.shape

(342, 513)

## the encoding an image
Is a binary tree represented as key value pairs where the key is the path to the node and the value is the log ratio between the number of +'s and the number of -'s.

In [13]:
EC.codes[0]

[('', 0.0),
 ('1', -0.02919355683826235),
 ('11', 0.33697384047685863),
 ('111', -3.231635678126578),
 ('1111', -3.417749595110774),
 ('11111', -3.621955427011411),
 ('11110', -3.2480626734607774),
 ('1110', -3.0745805597679414),
 ('11101', -12.660303253590426),
 ('11100', -2.3740849672373354),
 ('111001', -1.816905401201021),
 ('1110011', -1.3045687895157516),
 ('11100111', -1.2515248438148072),
 ('11100110', -1.3607480830913197),
 ('1110010', -2.9231659069674567),
 ('111000', -3.763624483441613),
 ('110', 1.016377019686501),
 ('1101', 1.0712378824462776),
 ('11011', -1.298762434037907),
 ('110111', -11.960595561841496),
 ('110110', -0.604578729145292),
 ('1101101', -4.871934635141988),
 ('1101100', 0.08317600230572066),
 ('11011001', 0.74037877570097),
 ('11011000', -2.567405970137284),
 ('11010', 1.716522067471926),
 ('110101', 1.203638242118677),
 ('1101011', 1.6594069155860547),
 ('11010111', -1.8006762736659667),
 ('11010110', 2.341595861883034),
 ('1101010', -0.02572387776473940

## The tree encodes each images into a fixed length array

In [14]:
EC.data

array([[ -3.52592641,  -2.90367983,  -2.38447261, ...,   0.        ,
          0.        ,   1.        ],
       [ -5.30869575,  -0.78522374,  -0.42167777, ...,   0.        ,
          0.        ,   0.        ],
       [ -6.10342913,  -1.71436068,  -1.07384508, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ -4.93607081,  -0.736417  ,  -0.09022156, ...,   0.        ,
          0.        ,   1.        ],
       [-12.48442025,  -2.11892989,  -1.42576334, ...,   0.        ,
          0.        ,   1.        ],
       [-12.48442025,  -2.51520938,  -1.84280409, ...,   0.        ,
          0.        ,   1.        ]])